In [ ]:
# Data manipulation and utilities
import os
import numpy as np
import glob
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

# Machine Learning Models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import tensorflow as tf
from sklearn.linear_model import ElasticNet

# Evaluation Metrics
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
# Handle warnings
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

In [ ]:
#read and merged csv file
path = "/kaggle/input/eyet4empathy-eye-movement-and-empathy-dataset/EyeT/EyeT" 
all_files = os.listdir(path)

df_list = [] 
for filename in all_files:
    if filename.endswith(".csv"):
        file_path = os.path.join(path, filename)
        df = pd.read_csv(file_path, low_memory=False)
        df_list.append(df)
        
cleaned_df = pd.concat(df_list)

In [ ]:
cleaned_df.shape

In [ ]:
cleaned_df.info

In [ ]:
cleaned_df.isnull().sum()

In [ ]:
#categorial columns
data_types = cleaned_df.dtypes
categorical_columns = []
for column in cleaned_df.columns:
    if data_types[column] == 'object':
        categorical_columns.append(column)

categorical_columns

In [ ]:
cleaned_df.describe()#descriptive statistics for a Data

In [ ]:
cleaned_df['Recording date'] = pd.to_datetime(cleaned_df['Recording date'])

In [ ]:
# Define the directory containing the CSV files
csv_directory = '/kaggle/input/eyet4empathy-eye-movement-and-empathy-dataset/EyeT/EyeT'

# Create an empty list to hold the resulting dataframes
dataframe_list = []

# Loop through each file in the directory
for filename in os.listdir(csv_directory):
    # Check if the file is a CSV file
    if filename.endswith('.csv'):
        # Read the CSV file
        current_dataframe = pd.read_csv(os.path.join(csv_directory, filename))
        numeric_columns = current_dataframe.select_dtypes(include=['float64', 'int64'])#Select columns

        # Extract the participant name from the first row
        participant_name = current_dataframe.loc[0, 'Participant name']

        # Drop the 'Participant name' column from the dataframe
        current_dataframe = current_dataframe.drop(columns=['Participant name'])

        # Add the participant name as a new column
        current_dataframe['Participant name'] = participant_name

        # Append the resulting dataframe to the list
        dataframe_list.append(current_dataframe)

# Concatenate all the resulting data into a single dataframe
final_df = pd.concat(dataframe_list)

In [ ]:
final_df.head(10)

In [ ]:
final_df.info()#Details

In [ ]:
final_df.shape

In [ ]:
final_df.isnull().sum()#null value

In [ ]:
final_df.describe()

In [ ]:
# Get a list of all the .csv files in the folder
filenames = glob.glob("/kaggle/input/eyet4empathy-eye-movement-and-empathy-dataset/EyeT/EyeT/*.csv")
Experiment_count = len(filenames)
print ("eye-gaze trajectories : ", Experiment_count)

In [ ]:
_ = final_df.hist(bins=50, figsize=(20,15))#histogram

In [ ]:
final_df

In [ ]:
# List of columns to clean
columns_to_clean = [
    'Pupil diameter right', 'Pupil diameter left', 'Eye position left X (DACSmm)',
    'Eye position left Y (DACSmm)', 'Eye position left Z (DACSmm)',
    'Eye position right X (DACSmm)', 'Eye position right Y (DACSmm)', 'Eye position right Z (DACSmm)',
    'Gaze point left X (DACSmm)', 'Gaze point left Y (DACSmm)', 'Gaze point right X (DACSmm)',
    'Gaze point right Y (DACSmm)', 'Gaze point X (MCSnorm)', 'Gaze point Y (MCSnorm)',
    'Gaze point left X (MCSnorm)', 'Gaze point left Y (MCSnorm)', 'Gaze point right X (MCSnorm)',
    'Gaze point right Y (MCSnorm)', 'Fixation point X (MCSnorm)', 'Fixation point Y (MCSnorm)',
    'Gaze direction left X', 'Gaze direction left Y', 'Gaze direction left Z',
    'Gaze direction right X', 'Gaze direction right Y', 'Gaze direction right Z'
]

# Clean columns: replace commas with periods and convert to float
for column in columns_to_clean:
    final_df[column] = final_df[column].astype(str).str.replace(',', '.').astype(float)

In [ ]:
format_string = "%Y-%m-%d"
final_df['Export date'] = pd.to_datetime(final_df['Export date'], errors='coerce')
final_df['Recording date'] = pd.to_datetime(final_df['Export date'], errors='coerce')
final_df['Recording date UTC'] = pd.to_datetime(final_df['Recording date UTC'], errors='coerce')
final_df['Recording start time'] = pd.to_datetime(final_df['Recording start time'], errors='coerce')
final_df['Recording start time UTC'] = pd.to_datetime(final_df['Recording start time UTC'], errors='coerce')

In [ ]:
participant_dict = {}
for i in range(1, 61):
    participant_dict[f"Participant{i:04d}"] = i

final_df["Participant name"] = final_df["Participant name"].map(participant_dict)

In [ ]:
final_df

In [ ]:
main_csv_score = pd.read_csv("/kaggle/input/eyet4empathy-eye-movement-and-empathy-dataset/19657323/Questionnaire_datasetIB.csv", encoding='latin1')
main_csv_score = main_csv_score.rename(columns={"Participant nr": "Participant name"})

In [ ]:
# Merge the two datasets based on the Participant name column
final_df = pd.merge(final_df, main_csv_score[['Participant name', 'Total Score extended']], on='Participant name', how='left')

# Set the timestamp column as the index of the DataFrame
final_df.set_index("Recording timestamp", inplace=True)

final_df.to_csv("eyetzip_data_with_score.csv", index=False)
print("CSV file 'eyetzip_data_with_score.csv' saved successfully.")

# Load and Combine Raw Data

In [ ]:
# Directory containing the TSV files with raw eye-tracking data
directory = "/kaggle/input/eyet4empathy-eye-movement-and-empathy-dataset/raw_data/raw_data"

# Initialize an empty list to store the loaded dataframes
df_list = []

# Retrieve a list of all TSV files in the specified directory
file_list = glob.glob(directory + '/*.tsv')

# Load all files into a list of DataFrames
for filepath in file_list:
    df = pd.read_csv(filepath, delimiter='\t', low_memory=False)
    df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
raw_data = pd.concat(df_list, ignore_index=True)
raw_data

In [ ]:
raw_data.info()

In [ ]:
raw_data.shape

In [ ]:
# Convert 'Recording date' and 'Export date' columns to datetime format, assuming day comes before month
raw_data['Recording date'] = pd.to_datetime(raw_data['Recording date'], dayfirst=True)
raw_data['Export date'] = pd.to_datetime(raw_data['Export date'], dayfirst=True)

# Define a format string for parsing datetime values
format_string = "%Y-%m-%d"

# Convert 'Export date', 'Recording date', 'Recording date UTC', 'Recording start time', and 'Recording start time UTC' columns
# to datetime format using the defined format string. Any errors during conversion are set to NaT (Not a Time).
raw_data['Export date'] = pd.to_datetime(raw_data['Export date'], format=format_string, errors='coerce')
raw_data['Recording date'] = pd.to_datetime(raw_data['Recording date'], format=format_string, errors='coerce')
raw_data['Recording date UTC'] = pd.to_datetime(raw_data['Recording date UTC'], format=format_string, errors='coerce')
raw_data['Recording start time'] = pd.to_datetime(raw_data['Recording start time'], format=format_string, errors='coerce')
raw_data['Recording start time UTC'] = pd.to_datetime(raw_data['Recording start time UTC'], format=format_string, errors='coerce')

In [ ]:
# Create an empty dictionary to store participant names and corresponding integer values
participant_dict = {}

# Iterate from 1 to 60 (inclusive)
for i in range(1, 61):
    # Format participant names with leading zeros ("Participant0001" to "Participant0060") and map to integers
    participant_dict[f"Participant{i:04d}"] = i

# Map the "Participant name" column using the participant_dict
raw_data["Participant name"] = raw_data["Participant name"].map(participant_dict)

In [ ]:
data_csv_score = pd.read_csv("/kaggle/input/eyet4empathy-eye-movement-and-empathy-dataset/19657323/Questionnaire_datasetIB.csv", encoding='latin1')
data_csv_score = data_csv_score.rename(columns={"Participant nr": "Participant name"})

In [ ]:
import pandas as pd

# Read the two datasets
raw_data = pd.read_csv("raw_data.csv")
data_csv_score = pd.read_csv("data_csv_score.csv")

# Define a generator function to merge the two datasets based on the Participant name column
def merge_dataframes(raw_data, data_csv_score):
    for _, row in raw_data.iterrows():
        score = data_csv_score.loc[data_csv_score['Participant name'] == row['Participant name'], 'Total Score extended'].iloc[0]
        row['Total Score extended'] = score
        yield row

# Merge the datasets using the generator function
merged_data = pd.DataFrame(merge_dataframes(raw_data, data_csv_score), columns=raw_data.columns)

# Set the timestamp column as the index of the DataFrame
merged_data.set_index("Recording timestamp", inplace=True)

# Save the merged dataset to a CSV file
merged_data.to_csv("tsv_data_with_score.csv", index=False)

print("CSV file 'tsv_data_with_score.csv' saved successfully.")

# #Load eyetzip_data_with_score.csv with empathy score for data analysis

In [ ]:
df1 = pd.read_csv('/kaggle/working/eyetzip_data_with_score.csv')
df1

In [ ]:
df1.describe()

#  Visual Analyis for eyetzip_data_with_score.csv

In [ ]:
df1['Recording duration'].hist()
plt.title('Recording Duration Distribution')
plt.xlabel('Recording Duration')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Scatter plot between `Gaze point X` and `Gaze point Y`
df1.plot.scatter(x='Gaze point X', y='Gaze point Y')
plt.title('Scatter Plot between Gaze point X and Gaze point Y')
plt.show()

# Correlation between `Gaze point X` and `Gaze point Y`
print(df1[['Gaze point X', 'Gaze point Y']].corr())

In [ ]:
# Create a count plot for the 'Eye movement type' column in the DataFrame df1
sns.countplot(x='Eye movement type', data=df1)
plt.title("Distribution of Eye Movement Types")
plt.xlabel("Eye Movement Type")
plt.ylabel("Count of Observations")

# Display plot
plt.show()

In [ ]:
# Set the figure size for the plot
plt.figure(figsize=(14, 7))

# Create a boxplot for the distribution of 'Eye movement type index' by 'Project name'
sns.boxplot(x=df1['Project name'], y=df1['Eye movement type index'])
plt.xlabel('Participant Name')
plt.ylabel('Eye Movement Type Index')
plt.title('Distribution of Eye Movement Type Index by Participant')
plt.xticks(rotation=90)

plt.show()

In [ ]:
# Select relevant columns (excluding non-numeric columns)
numeric_columns = ['Total Score extended', 'Gaze point X', 'Gaze point Y', 'Gaze event duration']
subset_df = df1[numeric_columns]

# Correlation analysis
correlation_matrix = subset_df.corr()

# Visualize correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# Differentiate into two groups based on project names

# Machine Learning Model

In [ ]:
# Select relevant columns for control group
control_selected_columns = ['Participant name', 'Recording duration',
                             'Pupil diameter left', 'Pupil diameter right',
                             'Eye position left X (DACSmm)', 'Eye position left Y (DACSmm)', 'Eye position left Z (DACSmm)',
                             'Eye position right X (DACSmm)', 'Eye position right Y (DACSmm)', 'Eye position right Z (DACSmm)',
                             'Gaze point left Y', 'Gaze point right X', 'Gaze point right Y',
                             'Gaze event duration', 'Fixation point X', 'Fixation point Y', 'Total Score extended', 'Gaze point X', 'Gaze point Y', 'Gaze event duration']

# Create a DataFrame with selected columns for control group
control_group_selected = control_group_data[control_selected_columns]

# Select relevant columns for test group
test_selected_columns = ['Participant name', 'Recording duration',
                         'Pupil diameter left', 'Pupil diameter right',
                         'Eye position left X (DACSmm)', 'Eye position left Y (DACSmm)', 'Eye position left Z (DACSmm)',
                         'Eye position right X (DACSmm)', 'Eye position right Y (DACSmm)', 'Eye position right Z (DACSmm)',
                         'Gaze point left Y', 'Gaze point right X', 'Gaze point right Y',
                         'Gaze event duration', 'Fixation point X', 'Fixation point Y', 'Total Score extended', 'Gaze point X', 'Gaze point Y', 'Gaze event duration']

# Create a DataFrame with selected columns for test group
test_group_selected = test_group_data[test_selected_columns]

In [ ]:
# Replace NaN values with 0 in control group dataframe
control_group_selected.fillna(0, inplace=True)

# Replace NaN values with 0 in test group dataframe
test_group_selected.fillna(0, inplace=True)

# Feature Engineering

In [ ]:
# Calculate Eye Movement Ratios for X, Y, and Z
control_group_selected['Eye_Position_Ratio_X'] = control_group_selected['Eye position left X (DACSmm)'] / (control_group_selected['Eye position right X (DACSmm)'] + 1e-6)
test_group_selected['Eye_Position_Ratio_X'] = test_group_selected['Eye position left X (DACSmm)'] / (test_group_selected['Eye position right X (DACSmm)'] + 1e-6)

control_group_selected['Eye_Position_Ratio_Y'] = control_group_selected['Eye position left Y (DACSmm)'] / (control_group_selected['Eye position right Y (DACSmm)'] + 1e-6)
test_group_selected['Eye_Position_Ratio_Y'] = test_group_selected['Eye position left Y (DACSmm)'] / (test_group_selected['Eye position right Y (DACSmm)'] + 1e-6)

control_group_selected['Eye_Position_Ratio_Z'] = control_group_selected['Eye position left Z (DACSmm)'] / (control_group_selected['Eye position right Z (DACSmm)'] + 1e-6)
test_group_selected['Eye_Position_Ratio_Z'] = test_group_selected['Eye position left Z (DACSmm)'] / (test_group_selected['Eye position right Z (DACSmm)'] + 1e-6)

In [ ]:
# Calculate Gaze Point Differences for X and Y
control_group_selected['Gaze_Point_Diff_X'] = control_group_selected['Gaze point left Y'] - control_group_selected['Gaze point right X']
test_group_selected['Gaze_Point_Diff_X'] = test_group_selected['Gaze point left Y'] - test_group_selected['Gaze point right X']

control_group_selected['Gaze_Point_Diff_Y'] = control_group_selected['Gaze point left Y'] - control_group_selected['Gaze point right Y']
test_group_selected['Gaze_Point_Diff_Y'] = test_group_selected['Gaze point left Y'] - test_group_selected['Gaze point right Y']

In [ ]:
# Drop columns used in feature engineering
columns_to_drop = ['Eye position left X (DACSmm)', 'Eye position right X (DACSmm)',
                   'Eye position left Y (DACSmm)', 'Eye position right Y (DACSmm)',
                   'Eye position left Z (DACSmm)', 'Eye position right Z (DACSmm)',
                   'Gaze point left Y', 'Gaze point right X', 'Gaze point right Y',
                   ]

control_group_selected.drop(columns=columns_to_drop, inplace=True)
test_group_selected.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# Define the input features (X) and target variable (y) for control group
X_control = control_group_selected.drop(columns=['Total Score extended', 'Participant name', 'Recording duration'])
y_control = control_group_selected['Total Score extended']
participants_control = control_group_selected['Participant name']

# Define the input features (X) and target variable (y) for test group
X_test = test_group_selected.drop(columns=['Total Score extended', 'Participant name', 'Recording duration'])
y_test = test_group_selected['Total Score extended']
participants_test = test_group_selected['Participant name']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, GlobalAveragePooling2D, Flatten, Input, concatenate
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

# Model: CNN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten

# Assuming X_control, y_control, X_test, y_test are your data
# You may need to preprocess your data, such as scaling it with StandardScaler
scaler = StandardScaler()
X_control_scaled = scaler.fit_transform(X_control)
X_test_scaled = scaler.transform(X_test)

# CNN model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_control.shape[1], 1)),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1)
])

cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Reshape input for CNN
X_control_reshaped = X_control_scaled.reshape((X_control_scaled.shape[0], X_control_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Training and evaluation for CNN - Control Group
print("CNN - Control Group:")
cnn_model.fit(X_control_reshaped, y_control, epochs=10, batch_size=32, validation_split=0.2)
predictions_control_cnn = cnn_model.predict(X_control_reshaped)
print_metrics("CNN", y_control, predictions_control_cnn)

# Training and evaluation for CNN - Test Group
print("CNN - Test Group:")
cnn_model.fit(X_test_reshaped, y_test, epochs=10, batch_size=32, validation_split=0.2)
predictions_test_cnn = cnn_model.predict(X_test_reshaped)
print_metrics("CNN", y_test, predictions_test_cnn)

# Model:RNN

In [ ]:
# RNN model
rnn_model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_control.shape[1], 1)),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1)
])

rnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Training and evaluation for RNN - Control Group
print("RNN - Control Group:")
rnn_model.fit(X_control_reshaped, y_control, epochs=10, batch_size=32, validation_split=0.2)
predictions_control_rnn = rnn_model.predict(X_control_reshaped)
print_metrics("RNN", y_control, predictions_control_rnn)

# Training and evaluation for RNN - Test Group
print("RNN - Test Group:")
rnn_model.fit(X_test_reshaped, y_test, epochs=10, batch_size=32, validation_split=0.2)
predictions_test_rnn = rnn_model.predict(X_test_reshaped)
print_metrics("RNN", y_test, predictions_test_rnn)

# Model: CapsNet Model

In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from keras import layers
import tensorflow as tf
from tensorflow import keras

# Custom Capsule Layer
class CapsuleLayer(keras.layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, input_shape=None, share_weights=True):
        super(CapsuleLayer, self).__init__()
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.input_spec = [keras.layers.InputSpec(shape=input_shape)]
        self.share_weights = share_weights

    def build(self, input_shape):
        if self.share_weights:
            if self.input_spec[0].shape[1]:
                self.kernel = self.add_weight(name='capsule_kernel',
                                              shape=(1, input_shape[0][-1], self.num_capsule * self.dim_capsule),
                                              initializer='glorot_uniform',
                                              trainable=True)
            else:
                self.kernel = self.add_weight(name='capsule_kernel',
                                              shape=(input_shape[0][-1], self.num_capsule * self.dim_capsule),
                                              initializer='glorot_uniform',
                                              trainable=True)

    def call(self, inputs, training=None):
        # Expand the input dimensions to 1 for correct reshaping
        inputs = tf.expand_dims(inputs, axis=-1)
        if self.share_weights:
            capsule_kernel = tf.tile(self.kernel, [tf.shape(inputs)[0], 1, 1])
        else:
            capsule_kernel = tf.tile(self.kernel, [1, 1, tf.shape(inputs)[1], 1])

        capsule_kernel = tf.expand_dims(capsule_kernel, -1)
        capsule_output = tf.keras.layers.Conv2D(filters=self.num_capsule * self.dim_capsule, kernel_size=1, strides=1, padding='VALID')(inputs)
        capsule_output = tf.expand_dims(capsule_output, -1)

        vote_output = tf.matmul(capsule_output, capsule_kernel, transpose_b=True)
        vote_output = tf.squeeze(vote_output, axis=-1)
        for _ in range(self.routings):
            routing_weight = tf.nn.softmax(vote_output, axis=2)
            vote_output = tf.zeros_like(vote_output)
            for i in range(self.num_capsule):
                output_for_i = tf.matmul(routing_weight[:, :, i : i + 1], capsule_output, transpose_a=True)
                output_for_i = tf.reshape(output_for_i, [-1, self.dim_capsule])
                vote_output += output_for_i
            vote_output = tf.expand_dims(vote_output, -1)
        return vote_output

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule, 1)

# Assuming X_control, y_control, X_test, y_test are your data
# You may need to preprocess your data, such as scaling it with StandardScaler
scaler = StandardScaler()
X_control_scaled = scaler.fit_transform(X_control)
X_test_scaled = scaler.transform(X_test)

capsnet_model = Sequential([
    CapsuleLayer(num_capsule=10, dim_capsule=16, routings=3, input_shape=(X_control.shape[1],)),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1)
])

capsnet_model.compile(optimizer='adam', loss='mean_squared_error')

# Training and evaluation for CapsNet - Control Group
print("CapsNet - Control Group:")
capsnet_model.fit(X_control_scaled, y_control, epochs=10, batch_size=32, validation_split=0.2)
predictions_control_capsnet = capsnet_model.predict(X_control_scaled)
print_metrics("CapsNet", y_control, predictions_control_capsnet)

# Training and evaluation for CapsNet - Test Group
print("CapsNet - Test Group:")
capsnet_model.fit(X_test_scaled, y_test, epochs=10, batch_size=32, validation_split=0.2)
predictions_test_capsnet = capsnet_model.predict(X_test_scaled)
print_metrics("CapsNet", y_test, predictions_test_capsnet)

# Model: GoogLeNet Model

In [ ]:
def create_googlenet_model(input_shape):
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(100, activation='relu'),
        Dense(50, activation='relu'),
        Dense(1)
    ])

    return model

input_shape_googlenet = (X_control_scaled.shape[1],)
googlenet_model = create_googlenet_model(input_shape_googlenet)

googlenet_model.compile(optimizer='adam', loss='mean_squared_error')

# Training and evaluation for GoogLeNet - Control Group
print("GoogLeNet - Control Group:")
googlenet_model.fit(X_control_scaled, y_control, epochs=10, batch_size=32, validation_split=0.2)
predictions_control_googlenet = googlenet_model.predict(X_control_scaled)
print_metrics("GoogLeNet", y_control, predictions_control_googlenet)

# Training and evaluation for GoogLeNet - Test Group
print("GoogLeNet - Test Group:")
googlenet_model.fit(X_test_scaled, y_test, epochs=10, batch_size=32, validation_split=0.2)
predictions_test_googlenet = googlenet_model.predict(X_test_scaled)
print_metrics("GoogLeNet", y_test, predictions_test_googlenet)